
# Rede básica para regressão em Keras

## Dataset


In [1]:
!pip install numpy==1.21.4
!pip install pandas==1.3.4
!pip install keras==2.1.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 53.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 62.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [2]:
import pandas as pd
import numpy as np

In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


- O objetivo é ppredizer a coluna **Strength**

### Analisar os dados.


In [4]:
concrete_data.shape

(1030, 9)

Pelo volume pequeno de dados precisa-se ter cuidado para não gerar *overfit*

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

### Separar a coluna que queremos prever das colunas de dados.

In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # remover Strength
target = concrete_data['Strength'] # Strength - coluna

<a id="item2"></a>


In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

- Normalizar os dados para estarem prontos para o uso na rede

In [10]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [12]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

<a id="item1"></a>


<a id='item32'></a>


## Keras


Usamos Keras com o Tensorflow por baixo

In [14]:
import keras

O modelo será uma rede simples totalmente conectada

In [15]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Definindo a Rede Neural


In [16]:

def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Treinar a rede com os dados


In [18]:
# instancia o modelo
model = regression_model()

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                450       
_________________________________________________________________
dense_5 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 51        
Total params: 3,051
Trainable params: 3,051
Non-trainable params: 0
_________________________________________________________________


 - Treinar a rede usando o método *FIT*.
 
 30% dos dados serão usados para validação


In [19]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)



Train on 721 samples, validate on 309 samples
Epoch 1/100


2023-09-19 18:16:06.335156: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2023-09-19 18:16:06.342130: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394315000 Hz
2023-09-19 18:16:06.342927: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a5a834d560 executing computations on platform Host. Devices:
2023-09-19 18:16:06.342974: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2023-09-19 18:16:06.471487: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

 - 0s - loss: 1668.9351 - val_loss: 1154.0608
Epoch 2/100
 - 0s - loss: 1538.1810 - val_loss: 1032.6876
Epoch 3/100
 - 0s - loss: 1316.6118 - val_loss: 844.5069
Epoch 4/100
 - 0s - loss: 987.1221 - val_loss: 607.4049
Epoch 5/100
 - 0s - loss: 618.1537 - val_loss: 380.8046
Epoch 6/100
 - 0s - loss: 344.2707 - val_loss: 242.0146
Epoch 7/100
 - 0s - loss: 235.3869 - val_loss: 197.6862
Epoch 8/100
 - 0s - loss: 217.4193 - val_loss: 184.2350
Epoch 9/100
 - 0s - loss: 206.5414 - val_loss: 176.3850
Epoch 10/100
 - 0s - loss: 198.1096 - val_loss: 173.1539
Epoch 11/100
 - 0s - loss: 191.2692 - val_loss: 168.8069
Epoch 12/100
 - 0s - loss: 185.6243 - val_loss: 165.6387
Epoch 13/100
 - 0s - loss: 180.2018 - val_loss: 164.2848
Epoch 14/100
 - 0s - loss: 175.9835 - val_loss: 161.3080
Epoch 15/100
 - 0s - loss: 171.6085 - val_loss: 159.4899
Epoch 16/100
 - 0s - loss: 168.4285 - val_loss: 157.7217
Epoch 17/100
 - 0s - loss: 164.8804 - val_loss: 157.0655
Epoch 18/100
 - 0s - loss: 162.1665 - val_loss:

In [28]:
model.predict([predictors_norm[:2]])


array([[69.65748 ],
       [70.186676]], dtype=float32)

In [29]:
target[:2]

0    79.99
1    61.89
Name: Strength, dtype: float64